In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

#### не помещается
model_name = "Vikhrmodels/Vikhr-Llama-3.2-1B-Instruct" 

# Загрузка токенайзера
tokenizer_llm = AutoTokenizer.from_pretrained(model_name)

# Загрузка модели и помещение на GPU
model_llm = AutoModelForCausalLM.from_pretrained(model_name).cuda()


def empty_gpu_cache():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.reset_accumulated_memory_stats()


# Проверка устройства
print(f"Модель размещена на: {next(model_llm.parameters()).device}")

Модель размещена на: cuda:0


In [2]:
def get_answer(prompt, context, model, tokenizer, tokens_number = 512, temp = 0.3, top_k = 50, top_p = 0.95):
    
    input_ids = tokenizer.apply_chat_template([{"role": "user", "content": context}, 
                                               {"role": "user", "content": prompt}],
                                              truncation=True,
                                          add_generation_prompt=False, padding=False, return_tensors="pt").to("cuda")
    with torch.no_grad():
        
        outputs = model.generate(
                input_ids=input_ids,
                max_new_tokens=tokens_number,
                do_sample=True,
                temperature=temp,
                top_k=top_k,
                top_p=top_p,
                pad_token_id=tokenizer.pad_token_id
        )

    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=False)[0]

    ans = decoded_output.split('<|start_header_id|>assistant<|end_header_id|>')
    answer = ans[-1].replace('<|eot_id|>', '').strip()
    return answer
    

In [3]:
# pip install transformers sentencepiece
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from numpy import dot
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd


tokenizer_bert = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model_bert = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
# model.cuda()  # uncomment it if you have a GPU

def embed_bert(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]  # вектор токена [CLS], который является агрегированной информацией о всём тексте
    #embeddings = model_output.pooler_output
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

# print(embed_bert_cls('привет мир', model, tokenizer))
# (312,)

/home/kolya/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
def cos_distance(e1, e2):
    return 1 - dot(e1, e2)/np.linalg.norm(e1)/np.linalg.norm(e2)

class Retrieval:
    def __init__(self, chunks, model, tokenizer):
        if isinstance(chunks, pd.DataFrame):
            self.chunks = list(chunks["chunks"])
        elif isinstance(chunks, list):
            self.chunks = chunks
        else:
            raise TypeError('чанки должны быть или списком или датафреймом')
        self.model = model
        self.tokenizer = tokenizer
        self.chunk_vecs = []
        for chunk in self.chunks:
            self.chunk_vecs.append(list(embed_bert(chunk, model, tokenizer)))
        self.chunk_vecs = np.array(self.chunk_vecs)

    def get_chunks(self):
        return self.chunks
    
    def find_context(self, question):
        q_emb = embed_bert(question, self.model, self.tokenizer)
        min_ind = 0
        distances = []
        for i in range(len(self.chunks)):
            distances.append(cos_distance(q_emb, self.chunk_vecs[i]))
        min_ind = np.argmin(distances)

        return self.chunks[min_ind], distances[min_ind], min_ind, distances

    def find_top_k_context(self, question, k):
        q_emb = embed_bert(question, self.model, self.tokenizer)
        min_ind = 0
        distances = []
        for i in range(len(self.chunks)):
            distances.append((cos_distance(q_emb, self.chunk_vecs[i]), i))

        distances.sort(reverse = False, key = lambda x: x[0])
        distances = distances[:k]
        indexes = list(map(lambda x: x[1], distances))
        dist = list(map(lambda x: x[0], distances))
        return indexes, dist
        
        

In [27]:
class RAG:
    def __init__(self, retr: Retrieval, model, tokenizer):
        self.retr = retr
        self.model_llm = model
        self.tokenizer_llm = tokenizer

    def _get_answer(self, prompt: str, context: str, model, tokenizer, tokens_number = 512, temp = 0.3, top_k = 50, top_p = 0.95):
        
        input_ids = tokenizer.apply_chat_template([{"role": "user", "content": context}, 
                                                   {"role": "user", "content": prompt}],
                                                  truncation=True,
                                              add_generation_prompt=False, padding=False, return_tensors="pt").to("cuda")
        with torch.no_grad():
            
            outputs = model.generate(
                    input_ids=input_ids,
                    max_new_tokens=tokens_number,
                    do_sample=True,
                    temperature=temp,
                    top_k=top_k,
                    top_p=top_p,
                    pad_token_id=tokenizer.pad_token_id
            )
    
        decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=False)[0]
    
        ans = decoded_output.split('<|start_header_id|>assistant<|end_header_id|>')
        answer = ans[-1].replace('<|eot_id|>', '').strip()
        return answer
    
    def answer(self, prompt: str, temp=0.3):
        indexes = self.retr.find_top_k_context(prompt,3)[0]
        context = ''
        for ind in indexes:
            context+=self.retr.chunks[ind]
            context+='\n\n'
            
        return self._get_answer(prompt, context, self.model_llm, self.tokenizer_llm, temp=temp)

In [28]:
df = pd.read_csv('chunks.csv')
retr = Retrieval(df, model_bert, tokenizer_bert)
rag = RAG(retr, model_llm, tokenizer_llm)

In [21]:
real_answers = []
for i in range(len(df)):
    real_answers.append(get_answer(df.loc[i, "questions"], df.loc[i, "chunks"], model_llm, tokenizer_llm, temp=0.1))

In [29]:
rag_answers = []
for i in range(len(df)):
    hypothesis = rag.answer(df.questions.loc[i], temp=0.1)
    rag_answers.append(hypothesis)

In [32]:
print(rag_answers[65])

Контактные данные комендантов общежитий, указанные в вашем вопросе, следующий:

- Общежитие №2: Комендант: Журкова Наталья Николаевна Завхоз: Синичкина Светлана Ивановна
- Общежитие №5: Комендант: Новожилова Валентина Михайловна

Общежитие №10 не указано в вашем запросе, поэтому я не могу предоставить информацию о коменданте этого общежития.


In [33]:
print(real_answers[65])

Контактные данные комендантов общежитий по указанным адресам:

- **Общежитие №2**:
  - Комендант: **Наталья Николаевна Журкова**
  - Телефон: +7-495-542-73-89

- **Общежитие №5**:
  - Комендант: **Валентина Михайловна Новожилова**
  - Телефон: +7-499-367-78-18


In [34]:
#Оценка LLM

In [35]:
from rouge_score import rouge_scorer

# Пример для вычисления ROUGE
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2'])

rouges = []


for i in range(len(df)):
    hypothesis = rag_answers[i]
    reference = real_answers[i]
    rouges.append(scorer.score(reference, hypothesis)['rouge1'].fmeasure)

print(np.mean(rouges))

0.4503223307775391


In [36]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score

# Убедитесь, что у вас есть токенизация
nltk.download('punkt_tab')


# Токенизация
rag_answers_tokenized = [nltk.word_tokenize(answer) for answer in rag_answers]
df_answers_tokenized = [nltk.word_tokenize(answer) for answer in real_answers]

# Вычисление BLEU и METEOR
bleu_scores = []
meteor_scores = []

for rag_answer, df_answer in zip(rag_answers_tokenized, df_answers_tokenized):
    # BLEU
    bleu_score = sentence_bleu([df_answer], rag_answer)
    bleu_scores.append(bleu_score)
    
    # METEOR
    meteor_score_value = meteor_score([df_answer], rag_answer)
    meteor_scores.append(meteor_score_value)

# Средние метрики
average_bleu = sum(bleu_scores) / len(bleu_scores)
average_meteor = sum(meteor_scores) / len(meteor_scores)

print(f"Average BLEU score: {average_bleu}")
print(f"Average METEOR score: {average_meteor}")


[nltk_data] Downloading package punkt_tab to /home/kolya/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/home/kolya/venv/lib/python3.12/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Average BLEU score: 0.17341084379625854
Average METEOR score: 0.3275126423446656


In [64]:
ans = rag.answer(input())
print(ans)

 Сколько месяцев будет выплачиваться стипендия от VK и какой размер стипендии?


Стипендия от VK выплачивается ежемесячно в течение 4 месяцев. Стоимость стипендии составляет 15 000 рублей в месяц.


In [40]:
list(df.questions)

['Какие образовательные профили подготовки реализует кафедра ФН-11 в рамках направления "Математика и компьютерные науки"?',
 'Какие научные достижения были у профессора П.А. Зилова и Н.П. Слугинова?',
 'Из каких частей состоит главный учебный корпус (ГУК) МГТУ и как они расположены?',
 'Как устроено сообщение между подразделениями и нумерация аудиторий в ГУК МГТУ?',
 'Когда был открыт учебно-лабораторный корпус (УЛК) МГТУ, и как проходило его строительство?',
 'Сколько длится обучение по программе машинного обучения и какой конкурс на место?\n\n',
 'Какие образовательные программы предлагает VK для студентов МГТУ им. Баумана?\n\n',
 'Каковы шансы трудоустройства выпускников Центра в компанию VK после окончания обучения на должность высококлассного Web-, ML- или мобильного разработчика с опытом работы с высоконагруженными системами?\n\n',
 'Какие возможности предоставляет проект для студентов?\n\n',
 'Могут ли подать заявку на треки основной программы обучения студенты очно-заочной фор